# Out of Band Protocol - Receiver

The out of band protocol allows agents to exchange messages without requiring a DIDComm channel. This can be used to establish a connection, request a presentation or issue a credential. 

The RFC is described [here](https://github.com/hyperledger/aries-rfcs/tree/master/features/0434-outofband)

Begin in the [sender notebook](http://localhost:8888/notebooks/Part%208%20-%20Out%20of%20Band%20Protocol.ipynb).

## 5. Initialise the Controller

In [1]:
%autoawait
import time
import asyncio

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

IPython autoawait is `on`, and set to use `asyncio`


## 6. Configure Connection Listeners

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connections_handler(payload):
    print("Connection Webhook : ", payload)
    
connections_listener = {
    "topic": "connections",
    "handler": connections_handler
}


agent_controller.register_listeners([connections_listener], defaults=True)

## 7. Receive Out of Band Invitation

**You should have copied this from the Sender notebook (Step 3)**

In [11]:
import json
oob_invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/out-of-band/1.0/invitation', '@id': 'c876d97e-ad5f-4a12-8229-768753856abb', 'request~attach': [{'@id': 'request-0', 'mime-type': 'application/json', 'data': {'json': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/offer-credential', '@id': '15a997ff-509a-46a7-abc1-d2a54d3fa596', '~thread': {}, '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'credential_preview': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]}, 'comment': 'create automated credential exchange', 'offers~attach': [{'@id': 'libindy-cred-offer-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJzY2hlbWFfaWQiOiAiUFFSWER4ZEdxUUdTWjh6NjlwNHhaUDoyOm9wZW5fbWluZWRfY29udHJpYnV0b3IyOjAuMC4xIiwgImNyZWRfZGVmX2lkIjogIlBRUlhEeGRHcVFHU1o4ejY5cDR4WlA6MzpDTDoxNDpkZWZhdWx0IiwgImtleV9jb3JyZWN0bmVzc19wcm9vZiI6IHsiYyI6ICI0ODg1MTAzNjgzMzcxNDE0NDQxNDgzNjAzNDI1NjAzMjc2MDg2MDQ1MTc0Njk4NDQzMzg4ODk2NzI4NDMwNDI0ODc4MjgwODQwOTUzNiIsICJ4el9jYXAiOiAiMTE5MTk3MTQwMzMzOTEzMDE2MTM2NzEwMzg3OTQ0NDY2NTEyODAwNjE2ODA4Mjk0MTEyMjg3MDExOTM1OTY3NzI5ODMyMjQ2ODE3Nzg5ODM0MTIyODUwMDMyMzQxOTMyNjI1MDAyNzIzNjgxNzE4MjA5MzQ1ODY3NTA1MDM3ODkyODU2MTUwNjQ4NDU0OTAwMDMxMjE0MjU4NTEyMzA1ODU5ODgxMzM3ODYyNjgyNjA2MDU2ODg4MDI3MDQ5NDkxNjQyNDk2OTMwNDg4MjUxMTM5NDI2ODgxODMyOTEzOTU2MzEzODQwOTgyMzU4NTE0NDE2ODE0Mzc4OTY1MjYyNzUxOTExNzY3ODM5NTI0MzI1MDk0OTIxNDEwMDM4MzA5MzEwNTE2OTg3ODIzMjM1NDUxOTMyMjE0Mjc0MTk4NTIwOTE0NjIxMzcyNzc5NTU2MzIxMjAzOTgyMjcyNDE5MjU4MTA0NjQ0NDE0Mjg2NTA5MzU0NTY4NTI4ODA0ODMyNjczODYwMTE5OTQzODg0ODE5NzQ1NDY4MDM4MDYyNjMzODU4MDc2Nzc1NzcwMzg5OTQ2MTE3NzE0ODM3NTQyMjY4NDA4NzA1NTgzNDQ3OTMxNDQ5NjI4MTkwODQxOTg5NzA5NjQ3NDI1ODQ4NjI1MjAyMTY3NTE4NjkxNjgwNTk1ODIyMjI0NDMzNTI3MjIwMTQyMDQxMDg0MzU4MjA4NzM5ODQ0OTk1ODE4OTE3MjAwNTQ2OTQxMjgxOTc4MDI0NTM0NDYyMDIzMDA5MTA1ODk4ODk1MDgxNzM5MTE0MTY5NjQzODAxNjg3MTIzODYyMzMxMjY2MTUzOTEyNDE2MzA2MDM2NzEyODY5Mjg5OTQ2NDIwOTU4Mjk5MTExNTgxNzU3MDE4MTk4NzQwMTU0NDMzMTU1OCIsICJ4cl9jYXAiOiBbWyJhZ2UiLCAiNDU3NzkxMTYwMzY4NTkwODQwMDMzMDAzOTg4NDYwMDg0NTA4NzgwMDg1MzU5NDYzMDk4MTEyOTMwODc3MjUyMzk2NTU5NjM4ODA5OTgzOTcwNTE0Mzg4MTIzNTA4MzkwNzAxNTcwODQxMjkxMDQ3MzIyODM4MzA2MDg4NTYxNDE1MTYwNDk0NjkwNDYwMzY1Njk0MTk0MjY4NTA0NjM1OTUzNTExMzg4MzE2MTUyNzk5MDg1ODY1ODM0OTc5NTAxNTEzNzc0NzYyNDE1Mzk2OTUyNzUxNzM2ODMxODA1OTQ5MjI3ODA0NDQxMzA1NTEzMjk5OTI0OTYxNDYwNjA1MTAxOTI1MjExOTM3NDg3MzkxMTY2NjM2MDU2NDgzNzcwMTYzOTM5MTg4MDA3NTYzNjg2Nzg2MDQxMDI0MzMxMDMxNDI2MDQ0OTY4Mzg1NDc4MTQzNDEzMTAyODM5NDQzNTUzNDcyNzMzMTUyMDM1OTA0NzQxMjA5Nzc5MjU4MTY0NDY3NTAxOTkxNjcwODkwNzY1NTg5NTY4NTU0OTc3NDk1MTQ0MDM2NDIyNTM3OTk2OTIzOTA1MjMwNTk1NzIyMzQ3ODMyNTg3MTY4NDczODc5OTIwNTYyMDQxNzkyNzMwMDY2ODY1NTcyODkwMTEzMDE0MTU0ODMzODcxMDg2OTE5ODMxNTAxNzAyOTYwMjQwODIzNzAyMzU2NTA5OTc0MjA2NzY2NzMxNTgyMTg2Nzg0ODA1MjEzMjc3NTMyNjYwNTM0OTMwNjUxMzcyMjYzMjc1MjE2MDY5MTg1NjU0NDUxMjU0NjExMTkxMjU4MjAwOTUzMDUwMDE2MTU3NDI5NTczOTE3MjMzMTEwNzY3NDQ1NDk4Njg3NzU1NTM2NDU0NDE3NDM3MTgxMzM2OTA2OTA1ODA3Il0sIFsibmFtZSIsICI0OTE2ODI5OTgyNzA1MjE3Mzg1NTU0ODY0NzM3OTIwNzAzNDMzNDk1NDQ1NTkxNjA2ODIwOTM5NDQ1NTY0Mzg1ODY3NTUwNDQ4NTgwMDY3NzMwMjA3NjE2NTQ5MDQ0NjA3NjIyNTI2Njg2OTIzODQ2ODM0NjU0OTAyMzM0ODg0MzAzMjc2ODk0NDQwNjgxNDYzOTM2NTQ3ODYyMjkzMzI0ODQxNDUyMTc3MDQzMjA3OTYzNzI5NzEwMTU2MDYwODkyMzg5OTQxMzU3ODE0MDE2MDYzNTkyMjY0MjQ3MjIyNTUyNTExOTY2NzQ1NDk5Mjk2MTMyNjM3ODI3MDM4MTg0MjcyMzU5NjM2OTc4NzM3OTA1MDE3MDg0MzYyMzI5NjUwMjY2OTU5NDI2NzQyMjk4NjIwMzcyNjQ3MjI2NDE0NjM0MDcxMjM4MzUwMzcwMTM5MDg0NTcwNTA0MDQ1ODI4NDIzNzU0MzI5MDExNDgyNDMyMzc2NDUyNTQzMDkyMjM1Mzk5ODk0ODcxNjU2MDg5MTYzMjEwNjQyOTUzMTcyNTQ0ODM5NDExNDAzNzA1MTg2ODE2MTI1Njg1OTAwOTI1MDc3MTEyMDU3MDYzODQ0MTQ3Mjg4NjIyMzQxNzMyMzc1NjQ0MzQ0NDAxNzg1NzY2NjA1NTAwNTYyOTMwODY0NzAxMzk3MDcxMDQ1MTc4NTgzNTMyNjg3NTA5MzI3ODIwNDIwNzA0NTYxNzYzMjIyNjYyNTc1MDc5MjMyMTM2ODc1MDA0NjgzMjc0ODEzNTQ0MTg2MjA4MjEwNTA2MjkwNDI2NzQ0NjU1MTM4MTk1Mjc4MjY3NTI2OTA4NTI1ODk0MDI2MTg0NTkyNzQwMjcwMTQ2MzkxNTQ4NTk4MDc1NjkyOTE0NjY0ODQzMDE5MTI4MTk4NzQiXSwgWyJtYXN0ZXJfc2VjcmV0IiwgIjQxODY3OTIwNzM1NzgzMjQwNDM4MzQwOTE3ODczMzMxNjAwMDA4Njc2MjA2MTMzMDcyMTEzMzg1OTQ3ODg1NTkzODkwNzg1Nzg5NzQxNzg1NjYyMzIwNzA0Nzc5Nzc2OTM0ODU4NTg4MDkwODU1NDc1ODU2MzQxODE2MjUxMDk4NTY2OTI3Mzg3OTg0NzcxMDIxNTIzNDQxOTkzODAxMDcxMTk1NTQ0NDY1NjA4NTk4NTE4ODA0MDAxOTA0MDM0NjMxMjgwMTMzMjA0MDY4NTM4NTkzNDM3MzQ5NTkxNTAwNzY1NTgwMzY4NTcyMTgxODk3MzcyOTg3NzYwMTc2NDg0MjA2NDQyMTczMzE3MjkxNzI1OTM5NzE0OTE0NzA4NTY3NTU5MzUxMjIyOTg4Nzk2NjM1MTc1OTAxMTEyMTYyOTcyNTQzNDc4NTEyNjYzMjg3Mzc0MjE2MzQ2NzI0MzI0NDQ2MzE3MDYxODEyNTM0Mjc4Mzc5NzU2OTg5MDE4MTA5NzE0NjMxMTMyODg1NzE1NzY1NjA0MzIzODc2MjE4MDc0NDM2NTY0NzcyNjY3MDI2Nzg4MTYyNDIyOTQ5MDkzNTYxNDEzNTAzMzkxNjU2NDM4MTMxNDI4ODQ0OTc4NjE5OTI0OTk3MTY4MDc0MDQ2MDA2OTQyMzcxMzAwNDc1NTYyMjM0NDc1MTkyNTcyNDUzNDk4OTkyODkxNDE1NDcwMTkyMjUyODgyMTU1MjQyNzg4MzE0ODU5MTIxNzc5NDgxNjQ4NTAzNjI5MjI0NzAzNDczNjA1MDI2MzY1ODYzMzUwODEwNTgxNjI0MDU4NTY4ODMzMzYxODc0MTczMTE4MzkxNjU1Njg2MzkxMTQ2NjU0NDA5NDg1ODUxMTg5OTA3MDM3OTg4NzM3Mzc4ODY5MjA4MSJdLCBbInNraWxsIiwgIjc5MjM0MTEzNDQ3OTQ3MzU4NTU4Mzg0NDA1MDQyMzI5NDQxODMyNDUxMDg3NTA0OTkwNzkwMTA4NzM2Mzg4NjQyOTkyNzU4Njg0NTY3Mzc4ODMzMTQ2NDMyMjMyODU3NDA4MjIwNjI5NjE4NTA2NDc0NDM2ODU5NzY1MTI3MjA2NzcyNTg3NTMyMzY5NzcxODU2MzQ0NjgxMDMyODIzMDY0NzA3NTI0NDkyNDA0MjcwMjg1ODI4NzYzNTA3MjcxNzk3NTgwOTIyMTYxNzA4MTg3OTgzNDkyOTAxNjQyNDc2NTk1OTUzMzE2Nzg4NzQzMDMyMDk3MDk1NDcyNDA1NzkyNzMxNDU1MjAwMTQyODU3OTg3MTI1ODY2NTU3Mjc0NjQ1MDA1MzM5ODg2MDM0MjQ4NzQzNjAxODY1NTYwNTU5MTkzODg5MTM2ODI4NTI5NDc3Mzg2NzM2NTE3OTg0MzU5NTk3ODMwNjIzMDQwMDk2OTIyMDQyNTg1ODI2Nzk5NjY0NzYwNzY5MTMxNTg2MzU0MzI3MDQ1MjEwMDgxMjg4OTY0NjUxNjc2MTQ2MTI0MTMxNDQyNTgzOTk5Njk5ODkwNTI3OTQ1ODE4Nzc1MDUwODg2Mzc2MTIxMjEzOTc2OTE1NjAyODUwMTM5MDI1NTk4MjU1MjI5MjQ0MzM2NTcxMzY5MjAxNjI0OTUwOTE1Nzc3MTE5NTY1Njg3MTYwMjY3MTE5MjMzMjY5MzgxMDQ0NjA4MzY4NTI3Mjk4MTYxODMwMzMxNTc0NDM4MTI4ODc3ODIwNDI2MDMzNDgzNDkxMDM1MTI2MDk3MTAwNTE5MDM3ODE2NjAyMTUxMTgxNzA5NDU5ODIyMTI3NjMxNzQ0NjQyNjczOTc2MzQ2MzIwNjY0NTc5NDc3OTI1NDgwMDg0NDA1NyJdXX0sICJub25jZSI6ICIxMTExNTQzNzQ0OTk5NTQ1NzQwNjk5MDAyIn0='}}]}}}], 'label': 'Alice', 'handshake_protocols': [], 'service': [{'id': '#inline', 'type': 'did-communication', 'recipientKeys': ['did:key:z6MknGQfGqcJrXtjKhuifSjvtDmZo2jjTVPv9QntJZ5GAUeB'], 'routingKeys': [], 'serviceEndpoint': 'http://192.168.65.3:8020'}]}
response = await agent_controller.oob.receive_invitation(oob_invite)
print(response)

ClientResponseError: 500, message='Internal Server Error', url=URL('http://bob-agent:8051/out-of-band/receive-invitation')

Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/helpers/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/helpers/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/helpers/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/python3.7/site

## 8. Get Connection

In [4]:
connection_id = response["connection_id"]
print(connection_id)

connection = await agent_controller.connections.get_connection(connection_id)

print(connection)
print("Connection State : ", connection["state"])

9528ed11-97cd-48d4-820d-cd682cc06c8c
{'connection_id': '9528ed11-97cd-48d4-820d-cd682cc06c8c', 'their_label': 'Alice', 'updated_at': '2020-11-02 09:46:23.512097Z', 'their_did': 'fMye9F8QS4eQoSB4QrDNj', 'initiator': 'external', 'request_id': 'c2acf256-b81a-4464-92f1-4b9c7d1d01ea', 'created_at': '2020-11-02 09:46:23.315114Z', 'state': 'response', 'my_did': 'E418SRJtbFgMGAWiXPao8f', 'routing_state': 'none', 'accept': 'auto', 'invitation_mode': 'once', 'invitation_key': 'GFttRWJAJ7cV1EDS1b3AJatE4nznL6wAH4tdHeh2VXQm'}
Connection State :  response


## 9. Send Trust Ping

This moves the connection to the active state

In [5]:
await agent_controller.messaging.trust_ping(connection_id, "active")

{'thread_id': '3252ae14-60d5-49d2-bff6-1d0cfef38f7e'}

Connection Webhook :  {'connection_id': '9528ed11-97cd-48d4-820d-cd682cc06c8c', 'their_label': 'Alice', 'updated_at': '2020-11-02 09:46:26.715763Z', 'their_did': 'fMye9F8QS4eQoSB4QrDNj', 'initiator': 'external', 'request_id': 'c2acf256-b81a-4464-92f1-4b9c7d1d01ea', 'created_at': '2020-11-02 09:46:23.315114Z', 'state': 'active', 'my_did': 'E418SRJtbFgMGAWiXPao8f', 'routing_state': 'none', 'accept': 'auto', 'invitation_mode': 'once', 'invitation_key': 'GFttRWJAJ7cV1EDS1b3AJatE4nznL6wAH4tdHeh2VXQm'}


## 16. Accept out of band credential exchange

In [9]:
response = await agent_controller.issuer.get_records()
print(response['results'])

[{'thread_id': '43f8b5f9-938d-436c-83ff-f49002261d47', 'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'initiator': 'external', 'auto_remove': True, 'state': 'offer_received', 'trace': False, 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'key_correctness_proof': {'c': '51333313275435416752644212964927833867957550369401001798454665886460543740128', 'xz_cap': '81445759547607113739656940556008534653915635636896474443493032797432510605951258693590206390637148306390208628634070355586587256912077873033082316669716279995050732952842174092275249285461859897797574713004851797308435260401565968431476097212153439131039797075265908747416465631114219078467896086147880465125505502207777076860381975802614712801866188268229652446268575806272090149382649304357873095056855260626411361389358940347686146894686309655628219418489209832271463620159654527787218470497720469893088761925741

In [7]:
import json
oob_invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/out-of-band/1.0/invitation', '@id': '9b7443b9-574a-4623-9555-bacaf3933de3', 'handshake_protocols': ['https://didcomm.org/connections/1.0/invitation', 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation'], 'request~attach': [{'@id': 'request-0', 'mime-type': 'application/json', 'data': {'json': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/offer-credential', '@id': 'f5be3593-60b0-43c1-9a0a-6ffb6ea09a04', '~thread': {}, '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'credential_preview': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]}, 'offers~attach': [{'@id': 'libindy-cred-offer-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJzY2hlbWFfaWQiOiAiUFFSWER4ZEdxUUdTWjh6NjlwNHhaUDoyOm9wZW5fbWluZWRfY29udHJpYnV0b3IxOjAuMC4xIiwgImNyZWRfZGVmX2lkIjogIlBRUlhEeGRHcVFHU1o4ejY5cDR4WlA6MzpDTDoxMjpkZWZhdWx0IiwgImtleV9jb3JyZWN0bmVzc19wcm9vZiI6IHsiYyI6ICI1MTk3MzU4OTUzNzM5NDM2ODcwODY4Nzg0MTc3NjY3Nzc5MDQwMTYwMTI4Mzc3ODM0MTU5MjIxOTAwNTI0NTM1NjQ3MTMyMDM1NjkzNSIsICJ4el9jYXAiOiAiODE5ODM5OTYwMDA1NTMxNzAwMjI5NjY1NTQwMDE0MDQ1OTA4NTA1MzcxNjAzMjMzNTcxNDE3NDE5MzU1NTE1ODQ1MjMzNjMxNjQyMzc3ODI3MTA4NDIyMTgwNTc1MjQzMjkwMDU4NDU5MzcwNTMwMDA3OTMxNDU4Mzg4NTA1NDEyNTg0MTgxNDU5NzQyNjY1OTE1Mzk1MjA4Njc2NzIzMzMyMTM4OTg5NTAyODc1MDMyMTc2MDU5NTA0Nzg0Njg2Nzk4Mzc4NzI5MjYzNzE4MDMzODg4NzA0ODQ5MjUwMDQ5NzAwNjc5MDE1MTc2NzEwMDI3MDE1OTQ4OTI0NDA5ODE1NDQyMjk0MDQ4NjI2NzgzOTA1MTI2MDM0NTg0NTE5Mjc5MTA5NTkxODU1NzY0ODAxNDc2MzM5OTM0MTY5NDY0OTg1ODIwNjYwNDEwNDgxODU2NDk0MzQ3NzQyNDI0MjI3NDYwNzY3OTM4MDY1MjkxMzQyMzc1MjE0NTQ0NjYzNDMwMDc1NTkwOTE4OTkxNTYwNTA5ODM0MjYwMjU0MzA3NTM1Mjg1MzAzMDM0NjE0NzU0MjU1OTQwODcwOTQyNjMzOTEyNjA4MzM5NzQyMTQ4NzU4OTAzNTI3Mzg2Njg3NTUxOTQxNzMzMzQyMzA0Nzk3MTEzMDAyNDE3OTEyNzc0MzI3NDEzMDI1MTMyNjcyMTgwMzk2NTk5MjAxNTg0MjgwMDIxNTYwMjk3MDMxMzU0NjMxNTYyMjM3NTI4OTQzOTYyMjQ2MTYzOTY5MDkzMDA5NDc3NDgzODU0MzA5MjQxMDM4NjEzOTAzMTc3NTQ0OTk1MjUxMzU0MjA1MTA4NTU4MDM2OTM5MzU2NTYxMDM0NTI5ODk2MDk0OTIyMDYwOTAyMTQ3NzY1MDA1NDgwNjA4MTI2IiwgInhyX2NhcCI6IFtbImFnZSIsICIxMjE5MzgzOTI2MDEzNTcwMjA3MTc4MzQ3MjgwNTI4MTk1MzM5MTk4MTAyMzg3NjUwODgyNzAzMzU0OTgwOTc3MzA4OTY5MTcyNTg5OTU4MDEzMzM4Nzg5NTIxMzg2NTEzOTQ1MTk2NDc4MTA4ODQ1Mzg3NjI5OTIwNzgwNTA5MTQwMDA2OTkyMjI3MjU3NTAyNzM2MDczMzM4MjY4ODY1NDU5ODU2OTg5NDE4MzA5ODQ2Njc4NjE3NDA0MDMyODMzNDMwNTQ0NTY5NjIwMTQyNjc1MDg1Njk3MTM2MTMyODI2OTc3MTgzMTAyMDA3NTQ2NDA3NTI2NDg3NzgwMzg1MjE0MzU4NjcwNjQ1ODQxOTY1NjIxMTk1NTY4MDEzMTgzMTMxNzQwMDM5OTM0NDEyNzAxNjI3NjQxODk5NTAxNjQyNzQwMjc0ODUzNTA4Njk2Mjk4MTc2MzA3NDE5MDQ2NDA5MDc1NjU2MTk0ODY2NTcxNDg4Mjk2MTcyOTYzNjM2NTE2OTc2NzM2ODExOTQ5NjMyNTE5NzEyNDQ3NTcyODQzMDQ3ODc5OTYzODQ4ODU5ODU3NTg4Mzk4ODQ0ODg0MDU0ODg4MTQyMjU2ODI3MTQ4NzU0NDg5MDYxMDA3NjU5OTMyMjI2NTEwOTY0NzAwMTUyMjgwODg1NjY5NTU5Mzc4NzQzNDgzNDg1NTAxNzg2MDM3Njk5ODE4MDc3MjA3Mzc5ODA0ODY2NDUyNjI5MTYzOTA4MTAxMTUyNTcxMjAyNTI3MDAwNjI4NTQzOTk1MTQyNzE4MzEzNDgyODIzMzYxMzUxMDAwNDg3OTM0NDU2ODIyOTc0Mzk2OTEyOTcwODYyMzMzMTY5ODA0NjA1NDk1ODQ2NjUwMTYwNDU4NzUyNjM5ODkyNDI1MzMxMDg0NjE4MzY2Il0sIFsic2tpbGwiLCAiNTQ5NjU1MDEzNjI3OTQ0ODExMDM1NjMwNTgyMzI4NjU3MDU5Nzk5NjkxNzk2Njg4NTg0NzA0MjEyODY1NTYwODk4NDI1NjU2MDg0ODA3MTQzMDgxNzkyMTUzNjM0ODU1MzEyNDU4MjU1MTE3MDQ4MTEwODMxNjk1ODMyMzkyODg2ODQ0NjUyMTI3NjQ0NTc3MDc3NDc3OTUxMjE4MDgzMDcyNzY4NjAzMDg3OTAyNzU2NDQ1MzIzODEzOTgyMDkxNDU4NDc4Njg5NDIwODA1ODc1MDAxODc2Mjc1MzA0Nzc5MTUyNTAwOTQ4MTE5MTc3NTAzMDM2MjI1OTU1ODYyOTYwMjIzNTI3Njk0MjY0MzE4ODIyMDE0NTI1MDA3NDQ1NjE2NDIyODU2ODE0NTM4MTM4ODc1MjcxNzAxMzMzNjQ4MTM4MjA5NTY3MDY5NjA3NDE3OTA0NjgwNzUyMjIxMjk3OTY4MTg0NjY3ODEzNDU3Mzg1Njg1MDA4NzM5MDgxMzU4MTcyNzYyOTcyMjM4MzAwMjA1OTI0MjkwNTU1ODgxMTY3NDY2NDQzNTgxMjcxNjcyNTAxNzYyMjA3MTk2MjQ2MDIwNjQ0Mzk0NjY4MjAxMTIzNzQyOTY2NTg1MDkyMjU5Nzg5MjYxMjQ5Mzg1MzQ0MzQ3ODE1NjU2MzA3NDA5MTYwNDI4MTc0NDQ1NjY5ODE3OTk4OTQ3MTM2MzA1MDE2MTk1MzU5MTg1NDE1ODQ5MTI2MjY0Njg2NDI0NzY4NDgzMTg1MjkyNDUzOTcyMzM0MTYyOTA1Nzg0OTY1NzA0NDYyMzkxMTA2NTQyMjgwODM4MTczMDI1OTI3MzA0NDY2NTA0NDcwODEwMTk2MzgxNzkwMjI4ODg2NTkxNTc1NDg2OTI0NTMwMDMwMDM2ODQ2MjMwIl0sIFsibWFzdGVyX3NlY3JldCIsICI3Mjg1MTI0NzYwNTcxNTkzNzkzNjM0NjUyMDA1ODM4OTA2OTg5ODc0NTM3MzgyNzE3NDA3OTM4OTY2NjcwNjc5MjA2NDM0MDY1NzIwMDY0NDcyMjc3ODM1NTg5MzU1OTMyOTE2NTUyNjE3MzAyMzQ2ODAxMDM5NjUyMjc0NDU0MzI4MzE3OTkwMzk4MjMxOTEzNDQ4OTY0MTg4MTA2NDIxMzM5MDgxMjQ3NDEwNDQ2Mzc1NzQwOTcyODE1MTQ1MDEzNTcyNjg3MTU1NzQwOTkzNTI1ODc5MzU3NTQ1MzU0MDE4ODk0NDU4OTE3NDg0OTY5MTUyNTE0ODQwMzEyNzMyMjkzMDM0MTcyMzIwOTQ1NDk1ODMwNjQ5MDM3NzU0NjQ0ODI3NTk4MzkxMTYzNDI3NjA2MzMzMTA3OTQzMDI3MTEwMjYwNTgzNjY5ODYzNzUwNzMwNzkzOTY4ODcxMjc4NTI2MDQzNDAzNjM5NDkzOTI0MjQ5NDA1NzUxMjgxNDYzNjAzNzk5NzAzODQ3NTA4MzU4OTg1MzYyNjg5NDcxMTgzNjAyMjIxMTQxNjc2OTE4Mzc5MjIzNTc5ODE4MTE1NDQ3OTM1OTEyMTU1NjgzODIzOTc4MDk0NzUxNDA2MDM1OTU1NTEwMjgyMjc3ODAyODU1OTAyODQ5OTg3OTE4MDM1OTQ3NjI3MjM3NTM4NDQxOTE5NTYxMTAyMzk1MTA0NTg0ODU2OTk0NzA5MzE5MTQzNTU5Mzg2OTM4MzY1MjI2NDYyODM4MDEwMzYwNDM3NTc3NzY3NDUyMTc3MjExOTMzNjQwODU1NDQxMDUzNzM3NjkyODkwNjMxOTA4NTAzMjQwNzU4MzkwODYyNTA0MzU5Nzc5NTI0MDI4MTI4MDczNDA1NDIzNDkxMzA1NzA4MDc2NSJdLCBbIm5hbWUiLCAiMTQ1MzY3Njk0NzM5NDk3Njc2MDY5NzY5OTAxMTA1NDM3MDMzNzg2MTkzOTY5NzE5NzE0ODI2MzgyMjI1NjY0MTEyNDI3MDY1Njc2MzUzMTk4MTQ1NTk0NzE0MDYyNTMwNzExNzE0Mzk4MjUwMDQwMzg2NDE4NDIxMzg3MjAxNDkxNTEwOTU3Nzg1MDU5NTYzMjg4OTY5OTk0ODQyNzAwMjc2NTc3MDcxNTkyMjM3MTM4NzgwNjQ1MjEwOTIzNDQ2Mjk2NTA5NzU5NTcwNzg5NzM2MzI5MTcyODQ0MjEyODk5OTA0MDEzNTU4ODY2MDY3NjAzMzc5NjI1NDgyNDA4NTQxMTk2MTc2Mjk1MDk2OTYzMjEyMDI5NzU3Nzg1NjE1NDIyODMwMzc2MjkxMjcyNTgxODI5MDc5NTIwNzkyNzk4OTQzODI1MjU2OTAwOTQxNDAyMzcwMTA3NzExNDQ3ODgxNjI2Mzg4ODY0NTk3NTk1NTU1MzkwMzMyMDA3MzY0MzQ0OTIwMjE0NDI0NjQyNzU4NzgyMzQ3NTY0MDQ4Nzc3Mzg4MTk0MzU5NjE5NDc3NDM0MDc2MzQ4Mjk1OTg2NTE0NzA5Mjk0NDE4OTA1ODkyNjQzMjE0ODg5ODc4NjIzNDgwNzcyNTgxMTIwMTE3MDg1Mjg4NTA5ODc4Mjg5ODIzMDc2MTg2NzkxNTY0ODUxMTE5Nzc4NDIyNzE3NzQ5NzA4NzI4OTc0MTQwNjQxMzYwNjE1NzgzNTAxOTUyMDEwMzIwMTI3OTA3NTk0MDQzMTk2NDA5Nzc0MDMwMTgxNjg3MDYzNTQ3Njc2NDU3OTc2MDc4MDA3NTExOTcwNTY3NDg0OTk3NDQ1NDEwMjIzNjMwNTEzNzk2MTY3ODkxNzI4NzcyMDkyMTY1MDE2MzQ2NTE3MjI4NSJdXX0sICJub25jZSI6ICI2MzI1MTExOTc4OTAzMDE5MzYwNzkwMjAifQ=='}}], 'comment': 'create automated credential exchange'}}}], 'label': 'Alice', 'service': [{'id': '#inline', 'type': 'did-communication', 'recipientKeys': ['did:key:z6MkvSe9hjU9WVCb72cpUgQHk5FGmmPanRLUviYS8rLCgML7'], 'routingKeys': [], 'serviceEndpoint': 'http://192.168.65.3:8020'}]}
response = await agent_controller.oob.receive_invitation(oob_invite)
print(response)

ClientResponseError: 500, message='Internal Server Error', url=URL('http://bob-agent:8051/out-of-band/receive-invitation')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/helpers/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/helpers/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/helpers/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/

## End Tutorial


In [7]:
await agent_controller.terminate()